In [1]:
import kfp 
import pandas as pd
from kfp.components import *

def read_iris():
    
    import numpy as np
    import pandas as pd
    from tensorflow.python.lib.io import file_io

    #from fs_gcsfs import GCSFS
    #gcsfs = GCSFS(bucket_name="mlopstest")
    #fs = gcsfs.GCSFileSystem(project='mlopsbw', token='google_default')
    
    df = pd.read_csv("gs://mlopstest/Iris.csv")
    print(df.head())
    df.drop('Species',axis=1, inplace=True)
    
    
#     with gcsfs.open("Iris.csv", "w") as gcs_file:
#         #gcs_file.write(local_file.read())
#         df.to_csv("gs://mlopstest/Iris.csv")
        
    print('to csv done')
    df = pd.read_csv("gs://mlopstest/Iris.csv")
    print(df.head())
    
    from tensorflow import gfile
    
    gcs_path = bucket + "/" + model_file
    
    if gfile.Exists(gcs_path):
        gfile.Remove(gcs_path)
    
    gfile.Copy("Iris.csv", gcs_path)
    
    return gcs_path
    

In [2]:
kfp_read_iris = kfp.components.func_to_container_op(func = read_iris, 
                                                          output_component_file = './read-iris-func.yaml',
                                                          packages_to_install = ['numpy==1.17.2',
                                                                                 'pandas==1.0.3','fs-gcsfs==1.2.0'])


In [3]:
import kfp.dsl as dsl

@dsl.pipeline(name='Read from Bucket Pipeline',description='read data from gs bucket')
def RIris_func():
    read_data_task = kfp_read_iris() 
    

In [4]:
pipeline_func = RIris_func
pipeline_filename = pipeline_func.__name__+'.pipeline.tar.gz'

import kfp.compiler as comp
comp.Compiler().compile(pipeline_func, pipeline_filename)